In [1]:
# import required libraries
import random
import datetime 
import numpy as np
import pandas as pd
from random import randrange
from sklearn import preprocessing

pd.options.mode.chained_assignment = None

In [2]:
def _random_date(start,date_count):##it is used to get random dates 
    
    current = start
    while date_count > 0:
        curr = current + datetime.timedelta(days=randrange(42))
        yield curr
        date_count-=1


def generate_sample_data(row_count=100):#generates random transactional dataset
   
    
    # sentinels:a sentinel value (also referred to as a flag value, trip value, rogue value, signal value, or dummy data) is a special value in the context of an algorithm which uses its presence as a condition of termination, 
    startDate = datetime.datetime(2016, 1, 1,13)
    serial_number_sentinel = 1000
    user_id_sentinel = 5001
    product_id_sentinel = 101
    price_sentinel = 2000
    
    
    # base list of attributes
    data_dict = {
    'Serial No': np.arange(row_count)+serial_number_sentinel,
    'Date': np.random.permutation(pd.to_datetime([x.strftime("%d-%m-%Y") 
                                                    for x in _random_date(startDate,
                                                                          row_count)]).date
                                  ),
    'User ID': np.random.permutation(np.random.randint(0,
                                                       row_count,
                                                       size=int(row_count/10)) + user_id_sentinel).tolist()*10,
    'Product ID': np.random.permutation(np.random.randint(0,
                                                          row_count,
                                                          size=int(row_count/10))+ product_id_sentinel).tolist()*10 ,
    'Quantity Purchased': np.random.permutation(np.random.randint(1,
                                                                  42,
                                                                  size=row_count)),
    'Price': np.round(np.abs(np.random.randn(row_count)+1)*price_sentinel,
                      decimals=2),
    'User Type':np.random.permutation([chr(random.randrange(97, 97 + 3 + 1)) 
                                            for i in range(row_count)])
    }
    
    # missing values are being introduced
    for index in range(int(np.sqrt(row_count))): 
        data_dict['Price'][np.argmax(data_dict['Price'] == random.choice(data_dict['Price']))] = np.nan
        data_dict['User Type'][np.argmax(data_dict['User Type'] == random.choice(data_dict['User Type']))] = np.nan
        data_dict['Date'][np.argmax(data_dict['Date'] == random.choice(data_dict['Date']))] = np.nan
        data_dict['Product ID'][np.argmax(data_dict['Product ID'] == random.choice(data_dict['Product ID']))] = 0
        data_dict['Serial No'][np.argmax(data_dict['Serial No'] == random.choice(data_dict['Serial No']))] = -1
        data_dict['User ID'][np.argmax(data_dict['User ID'] == random.choice(data_dict['User ID']))] = -101
        
    
    # create data frame
    df = pd.DataFrame(data_dict)
    
    return df
    

def describe_dataframe(df=pd.DataFrame()):#descriptive stats of a dataframe
    print("\n\n")
    print("*"*30)
    print("About the Data")
    print("*"*30)
    
    print("Number of rows::",df.shape[0])
    print("Number of columns::",df.shape[1])
    print("\n")
    
    print("Column Names::",df.columns.values.tolist())
    print("\n")
    
    print("Column Data Types::\n",df.dtypes)
    print("\n")
    
    print("Columns with Missing Values::",df.columns[df.isnull().any()].tolist())
    print("\n")
    
    print("Number of rows with Missing Values::",len(pd.isnull(df).any(1).nonzero()[0].tolist()))
    print("\n")
    
    print("Sample Indices with missing data::",pd.isnull(df).any(1).nonzero()[0].tolist()[0:5])
    print("\n")
    
    print("General Stats::")
    print(df.info())
    print("\n")
    
    print("Summary Stats::")
    print(df.describe())
    print("\n")
    
    print("Dataframe Sample Rows::")
    display(df.head(5))
    
def cleanup_column_names(df,rename_dict={},do_inplace=True):#rename columns if rename_dict is not passed
    
    if not rename_dict:
        return df.rename(columns={col: col.lower().replace(' ','_') 
                    for col in df.columns.values.tolist()}, 
                  inplace=do_inplace)
    else:
        return df.rename(columns=rename_dict,inplace=do_inplace)

def expand_user_type(u_type):# maps user types to user classes
   
    if u_type in ['a','b']:
        return 'new'
    elif u_type == 'c':
        return 'existing'
    elif u_type == 'd':
        return 'loyal_existing'
    else:
        return 'error'

In [3]:
df = generate_sample_data(row_count=1000)

In [4]:
describe_dataframe(df)




******************************
About the Data
******************************
Number of rows:: 1000
Number of columns:: 7


Column Names:: ['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


Column Data Types::
 Serial No               int32
Date                   object
User ID                 int64
Product ID              int64
Quantity Purchased      int32
Price                 float64
User Type              object
dtype: object


Columns with Missing Values:: ['Date', 'Price']


Number of rows with Missing Values:: 62


Sample Indices with missing data:: [0, 1, 3, 4, 6]


General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
Serial No             1000 non-null int32
Date                  969 non-null object
User ID               1000 non-null int64
Product ID            1000 non-null int64
Quantity Purchased    1000 non-null int32
Price                 969 non-null float64


C:\Users\SAPTADEEPA RAY\Anaconda3\lib\site-packages\ipykernel_launcher.py:78: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
C:\Users\SAPTADEEPA RAY\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead


         Serial No      User ID   Product ID  Quantity Purchased        Price
count  1000.000000  1000.000000  1000.000000         1000.000000   969.000000
mean   1455.340000  5495.981000   609.771000           20.697000  2417.211362
std     382.804775   353.405475   304.738022           11.776667  1630.606730
min      -1.000000  -101.000000     0.000000            1.000000    17.260000
25%    1229.750000  5235.000000   370.000000           10.000000  1164.670000
50%    1484.500000  5456.000000   602.500000           20.000000  2168.460000
75%    1742.250000  5808.000000   885.500000           31.000000  3462.940000
max    1999.000000  5994.000000  1096.000000           41.000000  8681.950000


Dataframe Sample Rows::


,Serial No,Date,User ID,Product ID,Quantity Purchased,Price,User Type
0,1000,NaN,-101,0,24,1534.78,n
1,1001,NaN,5848,274,15,2320.46,n
2,1002,2016-10-01,5235,114,19,346.84,n
3,1003,NaN,5741,252,37,1964.81,n
4,1004,NaN,5866,684,16,2238.82,n


In [5]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))#renaming data column

Dataframe columns:
['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


In [6]:
cleanup_column_names(df)

In [7]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['serial_no', 'date', 'user_id', 'product_id', 'quantity_purchased', 'price', 'user_type']


In [8]:
display(df.sort_values(['serial_no', 'price'], 
                         ascending=[True, False]).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
494,-1,2016-05-01,5183,228,11,6228.59,d
138,-1,2016-06-01,5060,392,20,5642.53,b
368,-1,2016-04-02,5061,125,9,4574.75,a
853,-1,2016-05-01,5814,601,1,4358.89,b
114,-1,2016-11-02,5008,1074,39,4071.93,c


In [9]:
display(df[['serial_no','date','user_id','user_type',
              'product_id','quantity_purchased','price']].head())

,serial_no,date,user_id,user_type,product_id,quantity_purchased,price
0,1000,NaN,-101,n,0,24,1534.78
1,1001,NaN,5848,n,274,15,2320.46
2,1002,2016-10-01,5235,n,114,19,346.84
3,1003,NaN,5741,n,252,37,1964.81
4,1004,NaN,5866,n,684,16,2238.82


In [10]:
# print 10 values from column at index 3
print(df.iloc[:,3].values[0:10])#Using Column Index

[   0  274  114  252  684  384  230  394  499 1075]


In [11]:
# print 10 values of quantity purchased
print(df.quantity_purchased.values[0:10])#Using Column Name

[24 15 19 37 16  1 33  7  7  1]


In [12]:
display(df.iloc[[10,501,20]])

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
10,1010,2016-10-01,5712,236,10,4972.82,n
501,1501,2016-01-15,5848,274,10,1178.66,a
20,1020,NaN,5445,370,12,2995.36,a


In [13]:
display(df.drop([0,24,51], axis=0).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,NaN,5848,274,15,2320.46,n
2,1002,2016-10-01,5235,114,19,346.84,n
3,1003,NaN,5741,252,37,1964.81,n
4,1004,NaN,5866,684,16,2238.82,n
5,1005,2016-07-01,5304,384,1,4172.04,n


In [14]:
display(df[df.quantity_purchased>25].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
3,1003,NaN,5741,252,37,1964.81,n
6,1006,NaN,5505,230,33,1405.51,n
11,1011,2016-01-23,5994,238,26,26.27,n
12,1012,2016-08-01,5706,206,34,2368.60,n
13,1013,2016-12-01,5176,394,27,1728.39,n


In [15]:
display(df[100:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
100,1100,2016-07-02,5368,439,30,5053.99,c
101,1101,2016-05-02,5848,274,7,3352.97,c
102,-1,2016-04-01,5235,114,15,2882.19,a
103,1103,2016-01-28,5741,252,34,1261.34,d
104,-1,2016-01-30,5866,684,13,3279.32,c


In [16]:
display(df[-10:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
990,1990,2016-01-13,5350,407,11,1169.19,b
991,1991,2016-02-01,5876,1087,8,1831.96,d
992,1992,2016-06-01,5084,1035,29,3549.12,b
993,-1,2016-01-31,5863,811,34,1341.49,a
994,1994,2016-06-02,5183,228,10,2148.24,c


In [17]:
df['date'] = pd.to_datetime(df.date)
# compare dtypes of the original df with this one
print(df.dtypes)

serial_no                      int32
date                  datetime64[ns]
user_id                        int64
product_id                     int64
quantity_purchased             int32
price                        float64
user_type                     object
dtype: object


In [18]:
df['user_class'] = df['user_type'].map(expand_user_type)
display(df.tail())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
995,1995,2016-01-28,5331,415,39,469.09,c,existing
996,1996,2016-03-01,5078,165,30,2791.01,c,existing
997,1997,2016-01-20,5142,977,22,50.23,d,loyal_existing
998,1998,2016-07-01,5833,370,38,2332.41,d,loyal_existing
999,1999,2016-03-02,5717,565,2,1675.16,a,new


In [19]:
display(df.select_dtypes(include=[np.number]).apply(lambda x: 
                                                        x.max()- x.min()))

serial_no             2000.00
user_id               6095.00
product_id            1096.00
quantity_purchased      40.00
price                 8664.69
dtype: float64

In [20]:
df['purchase_week'] = df[['date']].applymap(lambda dt:dt.week 
                                                if not pd.isnull(dt.week) 
                                                else 0)

In [21]:
display(df.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,NaT,-101,0,24,1534.78,n,error,0
1,1001,NaT,5848,274,15,2320.46,n,error,0
2,1002,2016-10-01,5235,114,19,346.84,n,error,39
3,1003,NaT,5741,252,37,1964.81,n,error,0
4,1004,NaT,5866,684,16,2238.82,n,error,0


In [22]:
df_dropped = df.dropna(subset=['date'])
display(df_dropped.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
2,1002,2016-10-01,5235,114,19,346.84,n,error,39
5,1005,2016-07-01,5304,384,1,4172.04,n,error,26
7,1007,2016-01-28,5139,394,7,1422.96,a,new,4
8,1008,2016-10-02,5976,499,7,4408.43,a,new,39
9,1009,2016-08-01,5035,1075,1,2478.97,a,new,31


In [23]:
df_dropped['price'].fillna(value=np.round(df.price.mean(),decimals=2),
                                inplace=True)

In [24]:
df_dropped['user_type'].fillna(method='ffill',inplace=True)

In [25]:
df_dropped['user_type'].fillna(method='bfill',inplace=True)

In [26]:
# sample duplicates
display(df_dropped[df_dropped.duplicated(subset=['serial_no'])].head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
104,-1,2016-01-30,5866,684,13,3279.32,c,existing,4
114,-1,2016-11-02,5008,1074,39,4071.93,c,existing,44
138,-1,2016-06-01,5060,392,20,5642.53,b,new,22
155,-1,2016-01-13,5411,375,1,188.55,a,new,2
180,-1,2016-01-29,5207,739,16,2817.05,b,new,4


Shape of df=(969, 9)


In [27]:
df_dropped.drop_duplicates(subset=['serial_no'],inplace=True)

In [28]:
display(df.dropna(thresh=3).head())
print("Shape of df={}".format(df.dropna(thresh=3).shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,NaT,-101,0,24,1534.78,n,error,0
1,1001,NaT,5848,274,15,2320.46,n,error,0
2,1002,2016-10-01,5235,114,19,346.84,n,error,39
3,1003,NaT,5741,252,37,1964.81,n,error,0
4,1004,NaT,5866,684,16,2238.82,n,error,0


Shape of df=(1000, 9)


In [29]:
display(pd.get_dummies(df,columns=['user_type']).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_class,purchase_week,user_type_a,user_type_b,user_type_c,user_type_d,user_type_n
0,1000,NaT,-101,0,24,1534.78,error,0,0,0,0,0,1
1,1001,NaT,5848,274,15,2320.46,error,0,0,0,0,0,1
2,1002,2016-10-01,5235,114,19,346.84,error,39,0,0,0,0,1
3,1003,NaT,5741,252,37,1964.81,error,0,0,0,0,0,1
4,1004,NaT,5866,684,16,2238.82,error,0,0,0,0,0,1


In [30]:
type_map={'a':0,'b':1,'c':2,'d':3,np.NAN:-1}
df['encoded_user_type'] = df.user_type.map(type_map)
display((df.tail()))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
995,1995,2016-01-28,5331,415,39,469.09,c,existing,4,2.0
996,1996,2016-03-01,5078,165,30,2791.01,c,existing,9,2.0
997,1997,2016-01-20,5142,977,22,50.23,d,loyal_existing,3,3.0
998,1998,2016-07-01,5833,370,38,2332.41,d,loyal_existing,26,3.0
999,1999,2016-03-02,5717,565,2,1675.16,a,new,9,0.0


In [31]:
display(df.sample(frac=0.2, replace=True, random_state=42).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
102,-1,2016-04-01,5235,114,15,2882.19,a,new,13,0.0
435,1435,2016-06-01,5651,805,41,304.57,c,existing,22,2.0
860,1860,2016-01-21,5748,279,24,4688.99,b,new,3,1.0
270,1270,2016-01-14,5558,1093,41,132.79,d,loyal_existing,2,3.0
106,1106,2016-01-20,5505,230,23,386.33,b,new,3,1.0


In [32]:
df_normalized = df.dropna().copy()
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df_normalized['price'].values.reshape(-1,1))
df_normalized['price'] = np_scaled.reshape(-1,1)

In [33]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
7,1007,2016-01-28,5139,394,7,0.162233,a,new,4,0.0
8,1008,2016-10-02,5976,499,7,0.506789,a,new,39,0.0
9,1009,2016-08-01,5035,1075,1,0.284108,a,new,31,0.0
22,1022,2016-02-01,5704,776,25,0.105422,a,new,5,0.0
23,1023,2016-01-22,5135,133,33,0.497607,b,new,3,1.0


In [34]:
print("Mean price of items purchased by user_type=a :: {}".format(df['price'][df['user_type']=='a'].mean()))

Mean price of items purchased by user_type=a :: 2328.320418410043


In [35]:
print(df['purchase_week'].value_counts())

3     152
4     151
2     105
22     55
5      55
44     52
26     50
9      48
39     47
31     45
53     43
35     43
13     39
0      31
17     31
48     29
18     24
Name: purchase_week, dtype: int64


In [37]:
print(df.groupby(['user_class'])['quantity_purchased'].sum())

user_class
error               666
existing           5141
loyal_existing     4747
new               10143
Name: quantity_purchased, dtype: int32


In [38]:
display(df.groupby(['user_class'])['quantity_purchased'].agg([np.sum,
                                                                np.mean,
                                                                np.count_nonzero]))
display(df.groupby(['user_class','user_type']).agg({'price':np.mean,
                                                        'quantity_purchased':np.max}))

,sum,mean,count_nonzero
user_class,,,
error,666,21.483871,31
existing,5141,20.320158,253
loyal_existing,4747,19.945378,238
new,10143,21.219665,478


price  quantity_purchased
user_class     user_type                                 
error          n          2751.284000                  41
existing       c          2307.636543                  41
loyal_existing d          2383.377261                  41
new            a          2328.320418                  41
               b          2618.230220                  41

In [39]:
display(df.groupby(['user_class','user_type']).agg({'price':{
                                                                'total_price':np.sum,
                                                                'mean_price':np.mean,
                                                                'variance_price':np.std,
                                                                'count':np.count_nonzero},
                                                   'quantity_purchased':np.sum}))  

C:\Users\SAPTADEEPA RAY\Anaconda3\lib\site-packages\pandas\core\groupby\generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


price                                     \
                         total_price   mean_price variance_price  count   
user_class     user_type                                                  
error          n            82538.52  2751.284000    1924.455682   31.0   
existing       c           560755.68  2307.636543    1589.618061  253.0   
loyal_existing d           548176.77  2383.377261    1598.302856  238.0   
new            a           556468.58  2328.320418    1534.813437  247.0   
               b           594338.26  2618.230220    1749.403216  231.0   

                         quantity_purchased  
                                        sum  
user_class     user_type                     
error          n                        666  
existing       c                       5141  
loyal_existing d                       4747  
new            a                       5099  
               b                       5044

In [40]:
display(df.pivot_table(index='date', columns='user_type', 
                         values='price',aggfunc=np.mean))

user_type,a,b,c,d,n
date,,,,,
2016-01-01,453.240000,1977.145000,3728.028000,1517.605714,NaN
2016-01-02,1865.648571,2297.220000,1713.970000,1905.198000,NaN
2016-01-13,2668.293333,3744.525000,2990.340000,NaN,NaN
2016-01-14,2649.856000,1845.630000,1234.380000,1927.014444,NaN
2016-01-15,1025.213333,2987.460000,2066.758000,3991.790000,NaN
2016-01-16,2027.185000,2256.322500,3746.810000,1822.420000,NaN
2016-01-17,3505.990000,2041.850000,3047.585000,1499.048571,NaN
2016-01-18,3055.315000,3324.535000,2477.291667,2156.735000,NaN
2016-01-19,2788.535000,1157.266667,2235.093333,3148.563750,NaN


In [41]:
print(df.stack())

0    serial_no                            1000
     user_id                              -101
     product_id                              0
     quantity_purchased                     24
     price                             1534.78
     user_type                               n
     user_class                          error
     purchase_week                           0
1    serial_no                            1001
     user_id                              5848
     product_id                            274
     quantity_purchased                     15
     price                             2320.46
     user_type                               n
     user_class                          error
     purchase_week                           0
2    serial_no                            1002
     date                  2016-10-01 00:00:00
     user_id                              5235
     product_id                            114
     quantity_purchased                     19
     price   